# Manual Way To Understand

In [14]:
from unstructured.partition.pdf import partition_pdf

In [16]:
raw_pdf_elements0 = partition_pdf(
    filename=r"C:\Users\csing\VSCode\Projects\Realtime_MultiModal_RAG\data0\2005_11401v4.pdf",
    strategy="hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types=["figure", "table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="extracted_data0"
)

Cannot set gray stroke color because /'P8' is an invalid float value
Cannot set gray non-stroke color because /'P8' is an invalid float value


Cannot set gray stroke color because /'P8' is an invalid float value
Cannot set gray non-stroke color because /'P8' is an invalid float value


In [ ]:
raw_pdf_elements0

In [ ]:
Header = []
Footer = []
Title = []
Text = []
ListItem = []
NarrativeText = []

for element in raw_pdf_elements0:
    if "unstructured.documents.elements.Header" in str(type(element)):
        Header.append(str(element))
    elif "unstructured.documents.elements.Footer" in str(type(element)):
        Footer.append(str(element))
    elif "unstructured.documents.elements.Title" in str(type(element)):
        Title.append(str(element))
    elif "unstructured.documents.elements.Text" in str(type(element)):
        Text.append(str(element))
    elif "unstructured.documents.elements.ListItem" in str(type(element)):
        ListItem.append(str(element))
    elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
        NarrativeText.append(str(element))

In [ ]:
NarrativeText = []
for element in raw_pdf_elements0:
    if "unstructured.documents.elements.NarrativeText" in str(type(element)):
        NarrativeText.append(str(element))

In [ ]:
raw_pdf_elements1 = partition_pdf(
    filename=r"C:\Users\csing\VSCode\Projects\Realtime_MultiModal_RAG\data1\Clouded_Judgement_8125.pdf",
    strategy="hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types=["figure", "table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="extracted_data1"
)

In [ ]:
raw_pdf_elements1

In [ ]:
Table = []
Title = []
Text = []
ListItem = []
NarrativeText = []

for element in raw_pdf_elements1:
    if "unstructured.documents.elements.Title" in str(type(element)):
        Title.append(str(element))
    elif "unstructured.documents.elements.Text" in str(type(element)):
        Text.append(str(element))
    elif "unstructured.documents.elements.ListItem" in str(type(element)):
        ListItem.append(str(element))
    elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
        NarrativeText.append(str(element))

In [ ]:
Image = []
for element in raw_pdf_elements1:
    if "unstructured.documents.elements.Image" in str(type(element)):
        Image.append(str(element))

# RAG Technique

In [17]:
def extract_labeled_elements(pdf_path, image_output_dir):
    raw_elements = partition_pdf(
        filename=pdf_path,
        strategy="hi_res",
        extract_images_in_pdf=True,
        extract_image_block_types=["figure", "table"],
        extract_image_block_to_payload=False,
        extract_image_block_output_dir=image_output_dir
    )

    labeled_data = []
    for element in raw_elements:
        label = type(element).__name__  # e.g., NarrativeText, Title, Header
        labeled_data.append((label, str(element)))

    return labeled_data

In [19]:
data0 = extract_labeled_elements(
    r"C:\Users\csing\VSCode\Projects\Realtime_MultiModal_RAG\data0\2005_11401v4.pdf",
    "extracted_data0"
)

data1 = extract_labeled_elements(
    r"C:\Users\csing\VSCode\Projects\Realtime_MultiModal_RAG\data1\Clouded_Judgement_8125.pdf",
    "extracted_data1"
)

all_data = data0 + data1

Cannot set gray stroke color because /'P8' is an invalid float value
Cannot set gray non-stroke color because /'P8' is an invalid float value


Cannot set gray stroke color because /'P8' is an invalid float value
Cannot set gray non-stroke color because /'P8' is an invalid float value


In [20]:
import os
from dotenv import load_dotenv

load_dotenv()

HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
os.environ["HUGGINGFACE_API_TOKEN"] = HUGGINGFACE_API_KEY

In [21]:
from langchain_core.prompts import PromptTemplate

text_prompt = PromptTemplate.from_template("""
You are an expert assistant specializing in text summarization for vector retrieval systems.

Your task: Create a comprehensive yet concise summary of the provided text that maximizes retrieval effectiveness.

Instructions:
- Identify the text's main topic, purpose, and key concepts
- Extract specific important facts, metrics, and key findings
- Note key themes, arguments, and relationships
- Include relevant context like dates, locations, or scope
- Use descriptive keywords that users might search for
- Structure the summary to be embedding-friendly for semantic search

Format: Provide a clear, informative summary in 2-4 sentences.

Text: {element}
""")

In [22]:
import requests

def summarization(text, max_length=512, min_length=50):
    url = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
    headers = {"Authorization": f"Bearer {HUGGINGFACE_API_KEY}"}
    params = {"max_length": max_length, "min_length": min_length, "do_sample": False}
    
    r = requests.post(url, headers=headers, json={"inputs": text, "parameters": params})
    return r.json()[0].get('summary_text', 'No summary generated') if r.ok else f"Error: {r.status_code} - {r.text}"


In [23]:
def summarize_text(text):
    
    try:
        if len(text) > 1000:
            text = text[:1000] + "..."

        result = summarization(text)
        return result.strip() if result else "No summary generated"
    except Exception as e:
        return f"Error: Could not summarize text - {str(e)}"

In [24]:
def summarize(data):

    summaries = []
    for i, (label, text) in enumerate(data, start=1):
        print(f"Original {label} {i}: {text[:100]}...")
        summary = summarization(text)
        print(f"Summary: {summary}\n")
        summaries.append(summary)

    print(f"\nTotal summaries generated: {len(summaries)}")
    return summaries


In [ ]:
narrative_data = [(label, text) for label, text in data1 if label == "NarrativeText"][:3]
summaries = summarize(narrative_data)

Original NarrativeText 1: JAMIN BALL...
Summary: CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of the U.S. for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots from across the globe.

Original NarrativeText 2: Every week I’ll provide updates on the latest trends in cloud software companies....
Summary: Every week I’ll provide updates on the latest trends in cloud software companies. Follow me on Twitter @jenniferjenson and @cnncloud for updates on new cloud software. I also write a weekly blog, Cloud Software Trends, which you can read here.

Original NarrativeText 3: One of the biggest debates in every platform shift is: do the incumbents win, or do...
Summary: One of the biggest debates in every platform shift is: do the incumbents win, or do the challengers win? This is the case in the U.S. for the first time in the history of the Democratic Party. The party is changing its name from the Democra